Test queries on chicago_bikes database 

In [2]:
import pymongo
from pymongo import MongoClient, UpdateOne
import json
# import pprint

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# Confirm list of databases
print(mongo.list_database_names())

['admin', 'chicago_bikes', 'config', 'local']


In [5]:
# Assign the database to a variable name
db = mongo.chicago_bikes

In [6]:
# Review collections in database
print(db.list_collection_names())

['divvy_rides_by_season', 'weather_daily', 'withoutStationName', 'divvy_rides_by_month', 'sig_prcp_yes', 'Top10StartStations', 'divvy_ridedata_merged', 'Top10Routes', 'sig_prcp_no', 'withLatLong', 'distinct_station_names', 'RouteDistance', 'withStationName', 'divvy_ridedata', 'Top10EndStations']


In [7]:
# Assign weather data to a variable 
weather_daily = db['weather_daily']

In [8]:
# Count the number of documents in weather data 
print(weather_daily.count_documents({}))

365


In [9]:
# Review a document in the weather_daily collection 
print(db.weather_daily.find_one())

{'_id': ObjectId('655f19d5f1c6af1dd0f57470'), 'date': '2022-01-01', 'cloud_cover': 90.0, 'precipitation': 0.18, 'min_temp': 33.22, 'max_temp': 42.1, 'morning_temp': 42.1, 'afternoon_temp': 38.43, 'evening_temp': 35.24, 'night_temp': 38.44, 'max_windspeed': 15.01}


In [10]:
# Convert precipitation field value to inches (currently in mm)
# Conversion factor: 1 mm = 0.03937 inches
mm_to_inches = 0.03937

In [11]:
# Iterate through documents and update values
for document in weather_daily.find():
    size_mm = document.get("precipitation")
    if size_mm is not None:
        size_inches = size_mm * mm_to_inches
        # Update the document with the new size value in inches
        weather_daily.update_one({"_id": document["_id"]}, {"$set": {"prcp_inches": size_inches}})

In [12]:
# Review a document in the weather_daily collection to confirm update
print(db.weather_daily.find_one())

{'_id': ObjectId('655f19d5f1c6af1dd0f57470'), 'date': '2022-01-01', 'cloud_cover': 90.0, 'precipitation': 0.18, 'min_temp': 33.22, 'max_temp': 42.1, 'morning_temp': 42.1, 'afternoon_temp': 38.43, 'evening_temp': 35.24, 'night_temp': 38.44, 'max_windspeed': 15.01, 'prcp_inches': 0.0070866}


In [13]:
# Drop 'size_inches' field as it has been replaced with 'prcp_inches' and values match 
# Specify the field you want to drop
field_to_drop = "size_inches"

In [14]:
# Update documents to unset the specified field
weather_daily.update_many({}, {"$unset": {field_to_drop: 1}})

In [15]:
# Review a document in the weather_daily collection to confirm drop
print(db.weather_daily.find_one())

{'_id': ObjectId('655f19d5f1c6af1dd0f57470'), 'date': '2022-01-01', 'cloud_cover': 90.0, 'precipitation': 0.18, 'min_temp': 33.22, 'max_temp': 42.1, 'morning_temp': 42.1, 'afternoon_temp': 38.43, 'evening_temp': 35.24, 'night_temp': 38.44, 'max_windspeed': 15.01, 'prcp_inches': 0.0070866}


In [16]:
# Define the threshold value for significant precipitation comparison 
threshold = 0.1

In [17]:
# Update documents to add the new field "sig_prcp"
# Add a new field 'sig_prcp' and populate 'yes' values if 'prcp_inches' is greather than or equal .1 inches
weather_daily.update_many(
    {"prcp_inches": {"$gte": threshold}},
    {"$set": {"sig_prcp": "yes"}}
)

weather_daily.update_many(
    {"prcp_inches": {"$lt": threshold}},
    {"$set": {"sig_prcp": "no"}}
)

In [18]:
# Review a document in the weather_daily collection to confirm update
print(db.weather_daily.find_one())

{'_id': ObjectId('655f19d5f1c6af1dd0f57470'), 'date': '2022-01-01', 'cloud_cover': 90.0, 'precipitation': 0.18, 'min_temp': 33.22, 'max_temp': 42.1, 'morning_temp': 42.1, 'afternoon_temp': 38.43, 'evening_temp': 35.24, 'night_temp': 38.44, 'max_windspeed': 15.01, 'prcp_inches': 0.0070866, 'sig_prcp': 'no'}


In [19]:
# Update documents to add the new field "avg_temp"
weather_daily.update_many(
    {},
    [
        {
            "$set": {
                "avg_temp": {
                    "$avg": ["$morning_temp", "$afternoon_temp", "$evening_temp"]
                }
            }
        }
    ]
)

In [20]:
# Review a document in the weather_daily collection to confirm update
print(db.weather_daily.find_one())

{'_id': ObjectId('655f19d5f1c6af1dd0f57470'), 'date': '2022-01-01', 'cloud_cover': 90.0, 'precipitation': 0.18, 'min_temp': 33.22, 'max_temp': 42.1, 'morning_temp': 42.1, 'afternoon_temp': 38.43, 'evening_temp': 35.24, 'night_temp': 38.44, 'max_windspeed': 15.01, 'prcp_inches': 0.0070866, 'sig_prcp': 'no', 'avg_temp': 38.59}


In [ ]:
# Close the MongoDB connection
mongo.close()